In [1]:
from pyspark.sql import functions as F

# ---- Parameters (edit these) ----
CUSTOMER_COUNT = 50000     # enough for a portfolio project
AGENT_COUNT    = 250
START_DATE     = "2014-01-01"
END_DATE       = "2024-12-31"

# Where to store "source files" in the Lakehouse Files area
CUSTOMERS_OUT_PATH = "Files/source/customers_csv"
AGENTS_OUT_PATH    = "Files/source/agents_csv"

StatementMeta(, 29975fb2-4ff5-4406-b541-91dfcbcc1f6b, 3, Finished, Available, Finished)

In [2]:
# Simple small lookup lists (keep it readable)
regions  = ["North", "South", "East", "West", "Central"]
channels = ["Branch", "Online", "Partner"]

agents = (
    spark.range(AGENT_COUNT)
    .withColumnRenamed("id", "agent_num")
    .withColumn("agent_id", F.format_string("AG%05d", F.col("agent_num") + F.lit(1)))
    .withColumn("agent_name", F.concat(F.lit("Agent "), F.col("agent_id")))
    .withColumn("region", F.element_at(F.array([F.lit(r) for r in regions]), (F.floor(F.rand() * len(regions)) + 1).cast("int")))
    .withColumn("channel", F.element_at(F.array([F.lit(c) for c in channels]), (F.floor(F.rand() * len(channels)) + 1).cast("int")))
    .withColumn("join_date", F.to_date(F.expr(f"date_add('{START_DATE}', cast(rand() * datediff('{END_DATE}','{START_DATE}') as int))")))
    .drop("agent_num")
)

display(agents)

StatementMeta(, 29975fb2-4ff5-4406-b541-91dfcbcc1f6b, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b3daf62a-a77e-4b4b-9c42-c12f50b0a9df)

In [3]:
states = ["ON", "QC", "BC", "AB", "MB", "SK", "NS", "NB"]  # tweak to your preference
cities = ["Toronto", "Montreal", "Vancouver", "Calgary", "Winnipeg", "Regina", "Halifax", "Moncton"]

risk_segments = ["Low", "Medium", "High"]
genders = ["Male", "Female", "Other"]

customers = (
    spark.range(CUSTOMER_COUNT)
    .withColumnRenamed("id", "cust_num")
    .withColumn("customer_id", F.format_string("C%07d", F.col("cust_num") + F.lit(1)))
    .withColumn("first_name", F.concat(F.lit("First"), (F.col("cust_num") % 5000).cast("string")))
    .withColumn("last_name",  F.concat(F.lit("Last"),  (F.col("cust_num") % 8000).cast("string")))
    .withColumn("full_name",  F.concat_ws(" ", F.col("first_name"), F.col("last_name")))
    .withColumn("gender", F.element_at(F.array([F.lit(g) for g in genders]), (F.floor(F.rand() * len(genders)) + 1).cast("int")))
    .withColumn("state",  F.element_at(F.array([F.lit(s) for s in states]),  (F.floor(F.rand() * len(states)) + 1).cast("int")))
    .withColumn("city",   F.element_at(F.array([F.lit(c) for c in cities]),  (F.floor(F.rand() * len(cities)) + 1).cast("int")))
    .withColumn("join_date", F.to_date(F.expr(f"date_add('{START_DATE}', cast(rand() * datediff('{END_DATE}','{START_DATE}') as int))")))
    # Age: 18–80 (roughly). We'll create birth_year based on random age.
    .withColumn("age", (F.floor(F.rand() * 63) + 18).cast("int"))
    .withColumn("birth_year", (F.year(F.current_date()) - F.col("age")).cast("int"))
    .withColumn(
        "risk_segment",
        F.when(F.rand() < 0.65, F.lit("Low"))
         .when(F.rand() < 0.90, F.lit("Medium"))
         .otherwise(F.lit("High"))
    )
    # Create an optional "preferred_agent_id" so later you can model sales channels
    .withColumn("preferred_agent_id", F.format_string("AG%05d", (F.floor(F.rand() * AGENT_COUNT) + 1).cast("int")))
    .drop("cust_num", "first_name", "last_name")
)

display(customers)

StatementMeta(, 29975fb2-4ff5-4406-b541-91dfcbcc1f6b, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 97e15e44-9910-492a-a2a2-0638ebb2280c)

In [4]:
# Write as CSV (source-style)
# Use header + overwrite for repeatable runs

(agents
 .coalesce(1)  # small dataset; makes it easy to view one file
 .write.mode("overwrite")
 .option("header", "true")
 .csv(AGENTS_OUT_PATH))

(customers
 .repartition(8)  # larger dataset: multiple files is fine
 .write.mode("overwrite")
 .option("header", "true")
 .csv(CUSTOMERS_OUT_PATH))

print("✅ Source data written to:")
print(f" - {AGENTS_OUT_PATH}")
print(f" - {CUSTOMERS_OUT_PATH}")

StatementMeta(, 29975fb2-4ff5-4406-b541-91dfcbcc1f6b, 6, Finished, Available, Finished)

✅ Source data written to:
 - Files/source/agents_csv
 - Files/source/customers_csv


In [5]:
df = spark.read.format("csv").option("header","true").load("Files/source/customers_csv/part-00007-0ac02b83-6404-4233-b0b0-bd03f4d403ee-c000.csv")
# df now is a Spark DataFrame containing CSV data from "Files/source/customers_csv/part-00007-0ac02b83-6404-4233-b0b0-bd03f4d403ee-c000.csv".
display(df)

StatementMeta(, 29975fb2-4ff5-4406-b541-91dfcbcc1f6b, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 69a04365-db45-4386-896f-5f64c76789e2)